#TASK 1
Perform an implementation for image enhancement in any image types (blurred image, dark image, bright image, low-contrast image, etc).

Create your notebook with name as "CVL_Assignment01" and upload it to your GitHub Project. You need to provide your GitHub link to the text editor in submission form.

#Program

## Import Library

In [ ]:
from google.colab import drive
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

## Load the data from Drive

In [ ]:
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/" #untuk menampilkan list dari drive

# load image
img = cv2.imread('/content/drive/My Drive/17340128845861609620620415790722.jpg')
cv2_imshow(img)

## Main program

### Resize image

In [ ]:
#resize image
# Dapatkan ukuran asli
(original_height, original_width) = img.shape[:2]

# Tentukan lebar baru yang diinginkan
new_width = 400

# Hitung rasio aspek dan tinggi baru
aspect_ratio = original_height / original_width
new_height = int(new_width * aspect_ratio)

# Ubah ukuran gambar dengan mempertahankan aspek rasio
aspect_ratio_resized_image = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

print(f"\n--- GAMBAR RESIZE DENGAN ASPEK RASIO (Lebar: {new_width}px) ---")
cv2_imshow(aspect_ratio_resized_image)

### Enhancement for Blurred Image

#### sharpening using

In [ ]:
# High pass kernel for sharpening :
kernel_sharpen = np.array([[0, -1, 0],
                           [-1, 5,-1],
                           [0, -1, 0]])
sharpened = cv2.filter2D(aspect_ratio_resized_image, -1, kernel_sharpen)
kernel_sharpen1 = np.array([[-1, -1, -1],
                           [-1, 9,-1],
                           [-1, -1, -1]])
sharpened1 = cv2.filter2D(aspect_ratio_resized_image, -1, kernel_sharpen)
#show_image("Enhanced (Sharpened)", sharpened)
print("====sharpened image using sharpened======")
cv2_imshow(sharpened)
print("====sharpened image using sharpened======")
cv2_imshow(sharpened1)

####sharpening using DOG filter

In [ ]:
# DoG Filter for sharpening:
sigma1 = 1
sigma2 = 3

g1 = cv2.GaussianBlur(aspect_ratio_resized_image, (0,0), sigma1)
g2 = cv2.GaussianBlur(aspect_ratio_resized_image, (0,0), sigma2)

dog = g1 - g2
dog_norm = cv2.normalize(dog, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Sharpen blurred image by adding DoG
enhanced = cv2.addWeighted(aspect_ratio_resized_image, 1.0, dog_norm, 0.05, 0)

# show_image("DoG Filter (Edge Map)", dog_norm)
# show_image("Enhanced with DoG", enhanced)
print("==== DoG Filter (edge map)======")
cv2_imshow(dog_norm)
print("====sharpened image using DoG Filter======")
cv2_imshow(enhanced)

####sharpening using laplacian filter

In [ ]:
# Laplacian Filter for sharpening:
lap = cv2.Laplacian(aspect_ratio_resized_image, cv2.CV_64F)
lap = cv2.convertScaleAbs(lap)
sharp2 = cv2.convertScaleAbs(aspect_ratio_resized_image - lap)
sharpened= cv2.addWeighted(aspect_ratio_resized_image, 1.0, lap, -0.4,0)
print("====sharpened image using Laplacian Filter======")
cv2_imshow(lap)
print("====sharpened image using Laplacian Filter======")
cv2_imshow(sharp2)
print("====sharpened image using Laplacian Filter======")
cv2_imshow(sharpened)

### Enhancement for Dark/Bright Image

In [ ]:
dark_img = cv2.convertScaleAbs(aspect_ratio_resized_image, alpha=0.5, beta=0)  # gambar digelapkan dulu
print("====dark image======")
cv2_imshow(dark_img)

bright_enhanced = cv2.convertScaleAbs(dark_img, alpha=1.5, beta=50)
print("====bright enhanced image======")
cv2_imshow(bright_enhanced)



bright_img = cv2.convertScaleAbs(aspect_ratio_resized_image, alpha=1.5, beta=70) #gambar diterangkan dulu
print("====bright image======")
cv2_imshow(bright_img)

dark_enhanced = cv2.convertScaleAbs(bright_img, alpha=0.7, beta=-50)
print("====dark enhanced image======")
cv2_imshow(dark_enhanced)

In [ ]:
# Convert to YCrCb (agar hanya Y/luminance yang diproses)
ycrcb = cv2.cvtColor(dark_img, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(ycrcb)

# 1. Histogram Equalization (HE)
y_he = cv2.equalizeHist(y)
he_img = cv2.merge([y_he, cr, cb])
he_img = cv2.cvtColor(he_img, cv2.COLOR_YCrCb2BGR)
print("====Enhanced - Histogram Equalization (HE)======")
cv2_imshow(he_img)

In [ ]:
# 2. Adaptive Histogram Equalization (AHE)
# Dengan tileGridSize besar supaya lebih jelas efeknya
clahe_ahe = cv2.createCLAHE(clipLimit=40.0, tileGridSize=(8,8))
y_ahe = clahe_ahe.apply(y)
ahe_img = cv2.merge([y_ahe, cr, cb])
ahe_img = cv2.cvtColor(ahe_img, cv2.COLOR_YCrCb2BGR)
print("====Enhanced - Adaptive Histogram Equalization (AHE)======")
cv2_imshow(ahe_img)

In [ ]:
# 3. Modified AHE (MAHE) -> implementasi dengan CLAHE normal
clahe_mahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
y_mahe = clahe_mahe.apply(y)
mahe_img = cv2.merge([y_mahe, cr, cb])
mahe_img = cv2.cvtColor(mahe_img, cv2.COLOR_YCrCb2BGR)
print("====Enhanced - Modified AHE (MAHE / CLAHE)======")
cv2_imshow(mahe_img)

### Enhancement for Bright Image

### Enhancement for Low Contrast Image

In [ ]:
low_contrast = cv2.convertScaleAbs(aspect_ratio_resized_image, alpha=0.5, beta=50) #gambar di low contrastkan dulu
print("====low contrast image======")
cv2_imshow(low_contrast)

# Histogram Equalization (YCrCb color space)
ycrcb = cv2.cvtColor(low_contrast, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(ycrcb)
y_eq = cv2.equalizeHist(y)
ycrcb_eq = cv2.merge([y_eq, cr, cb])
contrast_enhanced = cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2BGR)
print("====contrast enhanced image======")
cv2_imshow(contrast_enhanced)